In [1]:
#импортируем необходимые функции из библиотеки natasha
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    DatesExtractor,
    AddrExtractor,
    Doc
)

from yargy import Parser, rule, or_, and_
from yargy.predicates import gram, dictionary, gte, lte, type, eq
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display

In [2]:
#считываем текст из файла
file = open('olymp.txt')
text = file.read().splitlines()

In [3]:
#определяем необходимые функции для работы с текстом
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb) # морфологический разбор слова
syntax_parser = NewsSyntaxParser(emb) # анализ синтаксиса 
ner_tagger = NewsNERTagger(emb) #извлечение именованных сущностей

names_extractor = NamesExtractor(morph_vocab)
names_extractor('\n'.join(text))
dates_extractor = DatesExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [4]:
text

['Олимпи́йские игры  — крупнейшие международные комплексные спортивные соревнования, которые проводят раз в 4 года под эгидой Международного олимпийского комитета (МОК). Медаль, завоёванная на Олимпийских играх, считается одним из наивысших достижений в спорте. Традиция проведения Олимпийских игр, существовавшая в Древней Греции, зародилась как часть религиозного культа. Античные Олимпийские игры проводились в Олимпии, считавшейся у греков священным местом. От Олимпии произошло и название игр. В период с 776 года до н. э. по 393 год н. э. было проведено 292 Олимпиады. Проведение 293-й Олимпиады было отменено императором Римской империи Феодосием, а Олимпийские игры были запрещены как языческие. Современные Олимпийские игры были возрождены в конце XIX века французским общественным деятелем Пьером Кубертеном. Олимпийские игры, известные также как летние Олимпийские игры, проводили каждые 4 года, начиная с 1896 года, за исключением лет, пришедшихся на мировые войны. В 1924 году были учреж

In [14]:
from yargy import Parser, rule, or_, and_
from yargy.predicates import gram, dictionary, gte, lte, type, eq
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display



ParsStruct = fact(
    'Person',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first', 'last']
)


POST = morph_pipeline([
    'деятель',
    'барон',
    'поэт',
    'бог'
])

NAME = or_(
    rule(gram('Name').interpretation(Name.first.inflected()),
    gram('Surn').interpretation(Name.last.inflected())
    ).interpretation(Name),
    rule(gram('Name').repeatable().interpretation(Name.last.inflected())
    ).interpretation(Name))

ParsStruct = rule(
    POST.interpretation(ParsStruct.position.normalized()),
    NAME.interpretation(ParsStruct.name)
    ).interpretation(ParsStruct)

parser = Parser(ParsStruct)

In [15]:
matches = []
for tx in text:
    for match in parser.findall(tx):
        print(match.fact)

Person(position='деятель', name=Name(first='пьер', last='кубертен'))
Person(position='бог', name=Name(first=None, last='геракл'))
Person(position='бог', name=Name(first=None, last='зевс'))
Person(position='поэт', name=Name(first=None, last='панайотиса'))
Person(position='деятель', name=Name(first=None, last='евангелиса'))
Person(position='барон', name=Name(first='пьер', last='кубертен'))
Person(position='барон', name=Name(first='пьер', last='кубертен'))


In [16]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)

MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}

MONTH_NAME = dictionary(MONTHS)
DAY = and_(
    gte(1),
    lte(31)
)
YEAR = and_(
    gte(1700),
    lte(2022)
)
DATE = rule(
    DAY.interpretation(Date.day),
    MONTH_NAME.interpretation(Date.month.normalized()),
    YEAR.interpretation(Date.year).optional()
    ).interpretation(Date)

parser = Parser(DATE)

In [20]:
matches = []

for tx in text:
    for match in parser.findall(tx):
        print(match.fact)

Place(country='Греции', city=None)
Place(country='Греции', city=None)
Place(country='Франции', city=None)
Place(country='Греции', city=None)
Place(country='Греции', city=None)
Place(country='Франции', city=None)
Place(country='Германия', city=None)
Place(country='Франция', city=None)
Place(country=None, city='Афинах')
Place(country='Греции', city=None)
Place(country='Греции', city=None)
Place(country='Греции', city=None)
Place(country=None, city='Париже')
Place(country='Франция', city=None)
Place(country='США', city=None)
Place(country=None, city='Париже')
Place(country=None, city='Афинах')
Place(country='Греция', city=None)
Place(country=None, city='Париже')
Place(country=None, city='Афинах')
Place(country='СССР', city=None)
Place(country=None, city='Москве')
Place(country='России', city=None)
Place(country=None, city='Москва')
Place(country='СССР', city=None)
Place(country='Югославия', city=None)
Place(country='Россия', city=None)


In [18]:
Place = fact(
    'Place',
    ['country', 'city']
)

COUNTRY = {
    'Россия',
    'Германия',
    'Греция',
    'СССР',
    'Китай',
    'Югославия',
    'США',
    'Франция'
    
}
COUNTRY_NAME = dictionary(COUNTRY)

CITY = {
    'Москва',
    'Берлин',
    'Париж',
    'Сент-Луис',
    'Афины'
}
CITY_NAME = dictionary(CITY)

PLACE = or_(
    rule(
      COUNTRY_NAME.interpretation(Place.country)
    ),
    rule(
      CITY_NAME.interpretation(Place.city)
    ),
    ).interpretation(Place)

parser = Parser(PLACE)

In [19]:
matches = []

for tx in text:
    for match in parser.findall(tx):
        print(match.fact)

Place(country='Греции', city=None)
Place(country='Греции', city=None)
Place(country='Франции', city=None)
Place(country='Греции', city=None)
Place(country='Греции', city=None)
Place(country='Франции', city=None)
Place(country='Германия', city=None)
Place(country='Франция', city=None)
Place(country=None, city='Афинах')
Place(country='Греции', city=None)
Place(country='Греции', city=None)
Place(country='Греции', city=None)
Place(country=None, city='Париже')
Place(country='Франция', city=None)
Place(country='США', city=None)
Place(country=None, city='Париже')
Place(country=None, city='Афинах')
Place(country='Греция', city=None)
Place(country=None, city='Париже')
Place(country=None, city='Афинах')
Place(country='СССР', city=None)
Place(country=None, city='Москве')
Place(country='России', city=None)
Place(country=None, city='Москва')
Place(country='СССР', city=None)
Place(country='Югославия', city=None)
Place(country='Россия', city=None)
